<a href="https://colab.research.google.com/github/junieberry/ML-PerfectGuide/blob/main/04.%20%EB%B6%84%EB%A5%98/05%20GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GBM의 개요 및 실습

부스팅 알고리즘은 여러 개의 약한 학습기를 순차적으로 학습-예측하며 잘못 예측한 데이터에 가중치 부여를 통해 부스팅 수행

1. AdaBoost

    - 오류 데이터에 가중치 부여
    - 가중치는 손으로 부여하는듯
2. 그래디언트 부스트

    - 가중치 업데이트를 경사 하강법으로

In [1]:
# 중복된 feature 이름 변경
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    
    feature_dup_df = feature_dup_df.reset_index()
    print(feature_dup_df)
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [3]:
import pandas as pd

def get_human_dataset():

    # 공백으로 구분
    feature_name_df = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/features.txt', sep='\s+', header=None, names=['column_index', 'column_name'])

    # 중복된 피처명 수정
    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    # Dataframe한테 피처명 칼럼으로 주기 위해서 리스트로 변경
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()

    # 피처 데이터 세트링 라벨 데이터 세트
    x_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/X_train.txt',sep='\s+', names=feature_name )
    x_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/X_test.txt',sep='\s+', names=feature_name)
    
    y_train = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('/content/drive/MyDrive/UCI HAR Dataset/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 반환
    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = get_human_dataset()

     index  dup_cnt
0        0        0
1        1        0
2        2        0
3        3        0
4        4        0
..     ...      ...
556    556        0
557    557        0
558    558        0
559    559        0
560    560        0

[561 rows x 2 columns]


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

x_train, x_test, y_train, y_test = get_human_dataset()

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(x_train , y_train)
gb_pred = gb_clf.predict(x_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

     index  dup_cnt
0        0        0
1        1        0
2        2        0
3        3        0
4        4        0
..     ...      ...
556    556        0
557    557        0
558    558        0
559    559        0
560    560        0

[561 rows x 2 columns]
GBM 정확도: 0.9386
GBM 수행 시간: 808.5 초 


일반적으로 랜덤 포레스트보다 예측 성능이 좋음
- **수행 시간이 오래 걸림**
- 하이퍼 파라미터 튜닝 필요

### GBM 하이퍼파라미터 및 튜닝

1. `loss` : 경사 하강법 비용 함수
2. `learning rate`
3. `n_estimaators` : weak learner 개수
4. `subsample` :샘플링 비율 (1이면 전체 데이터 기반으로 학습)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}
grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(x_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
gb_pred = grid_cv.best_estimator_.predict(x_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))